## 1. Install necessary libraries

In [ ]:
# pip install langchain langchain-community pypdf sentence-transformers ollama

In [ ]:
# ! pip install sentence_transformers

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

## --- 1. Data Preparation ---
Replace with the path to your technical book

In [2]:

book_path = "C:/Users/amith/Documents/genai_proj1/unix programmers manual.pdf"
loader = PyPDFLoader(book_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=300,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(documents)
print(f"Split book into {len(chunks)} chunks.")

Split book into 336 chunks.



## --- 2. Create Embeddings and Vector Store ---
Using a local embedding model

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

C:\Users\amith\AppData\Local\Temp\ipykernel_37256\3367226509.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\amith\anaconda3\envs\torch_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create a FAISS vector store from the chunks

In [4]:
print("Creating vector store (this might take a moment)...")
vectorstore = FAISS.from_documents(chunks, embeddings)
print("Vector store created.")


Creating vector store (this might take a moment)...
Vector store created.


* You can save and load the vectorstore to avoid re-embedding every time
* vectorstore.save_local("faiss_index")
* vectorstore = FAISS.load_local("faiss_index", embeddings)

## --- 3. Implement RAG ---
* Initialize Ollama LLM (make sure Ollama server is running and model is pulled)
* Example: ollama pull gemma:2b

In [5]:
llm = Ollama(model="gemma:2b") # Replace with your chosen Ollama model


C:\Users\amith\AppData\Local\Temp\ipykernel_37256\657799462.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma:2b") # Replace with your chosen Ollama model


In [6]:
# Create a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Retrieve top 3 relevant chunks

In [8]:
# Define your RAG prompt template
rag_prompt_template = """Use the following context to answer the question at the end.
If you don't know the answer, state that you don't know, and do not make up an answer.

Context:
{context}

Question: {question}

Answer:"""
RAG_PROMPT = PromptTemplate(
    template=rag_prompt_template,
    input_variables=["context", "question"]
)


In [9]:

# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 'stuff' combines all retrieved documents into one prompt
    retriever=retriever,
    return_source_documents=True, # Optional: to see which chunks were used
    chain_type_kwargs={"prompt": RAG_PROMPT}
)


## --- 4. Chatbot Interaction (Basic CLI) ---

In [ ]:
print("\nChatbot initialized! Ask questions about your technical book.")
print("Type 'exit' to quit.")

while True:
    query = input("\nYour question: ")
    if query.lower() == 'exit':
        break

    result = qa_chain.invoke({"query": query})
    print("\nChatbot Answer:")
    print(result["result"])
    # Optional: print source documents
    # print("\nSource Documents:")
    # for doc in result["source_documents"]:
    #     print(f"- {doc.metadata.get('source', 'Unknown source')}: {doc.page_content[:150]}...")


Chatbot initialized! Ask questions about your technical book.
Type 'exit' to quit.

Chatbot Answer:
The context does not provide any information about insurance, double down, or the random number generator, so I cannot answer this question from the context.

Chatbot Answer:
The context does not provide a list of important commands, so I cannot answer this question from the provided context.

Chatbot Answer:
The context does not specify what mkdir is, so I cannot answer this question from the context.

Chatbot Answer:
The context does not provide information about copy commands, so I cannot answer this question from the provided context.

Chatbot Answer:
Sure, here is the answer to the question:

The context does not define what a directory is, so I cannot answer this question from the context.

Chatbot Answer:
Sure, here's the answer to your question:

The context describes a file system format and its structure. It mentions that a file system consists of a super block, a free list of

Chatbot initialized! Ask questions about your technical book.
Type 'exit' to quit.

Chatbot Answer:
The context does not provide any information about what unix is, so I cannot answer this question from the context.

Chatbot Answer:
The context does not provide any information about important commands to copy files, so I cannot answer this question from the context.

Chatbot Answer:
Sure, here's the answer to the question:

The context does not specify what the ls command is, so I cannot answer this question from the provided context.

Chatbot Answer:
The context does not specify what cp is, so I cannot answer this question from the context.

Chatbot Answer:
A directory is a special type of file that contains other files and or directories. A directory is indicated by a bit in the file's i-node entry, and it can have up to 16 entries. The first two words of each entry contain the i-number and filename, respectively, followed by a null-padded string of up to 14 characters. The first two words of each entry are for the directory itself, and the second is for the parent directory.

Chatbot Answer:
The context does not mention anything about "hi", so I cannot answer this question from the context.

Chatbot Answer:
The context does not provide any information about more, so I cannot answer the question.

Chatbot Answer:
The context does not specify what mkdir is, so I cannot answer this question from the context.